In [2]:
import pandas as pd
!pip install backtesting
from backtesting import Backtest, Strategy


     ------------------------------------ 175.5/175.5 kB 960.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173970 sha256=ec6e94059da980115002757aaa261a29732d17922207b3a3142f6fd452cb70a7
  Stored in directory: c:\users\rozir\appdata\local\pip\cache\wheels\e7\89\21\ebf3b5c7680f923554bb883380c53a28dedb085845df14ceb1
Successfully built backtesting


E:\Anaconda\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [7]:
import pandas_ta as ta
import pytz


In [4]:
import MetaTrader5 as mt
from datetime import datetime
mt.initialize()
account_info = mt.account_info()._asdict()
account_info

{'login': 122940720,
 'trade_mode': 0,
 'leverage': 2000,
 'limit_orders': 1024,
 'margin_so_mode': 0,
 'trade_allowed': True,
 'trade_expert': True,
 'margin_mode': 2,
 'currency_digits': 2,
 'fifo_close': False,
 'balance': 25.62,
 'credit': 0.0,
 'profit': -2.68,
 'equity': 22.94,
 'margin': 1.04,
 'margin_free': 21.9,
 'margin_level': 2205.7692307692305,
 'margin_so_call': 60.0,
 'margin_so_so': 0.0,
 'margin_initial': 0.0,
 'margin_maintenance': 0.0,
 'assets': 0.0,
 'liabilities': 0.0,
 'commission_blocked': 0.0,
 'name': 'Demo-account',
 'server': 'Exness-MT5Trial7',
 'currency': 'USD',
 'company': 'Exness Technologies Ltd'}

In [19]:
def get_15m_candles(symbol, start_date, end_date):
   
    if not mt.initialize():
        print("initialize() failed")
        mt.shutdown()
        return None

    timeframe = mt.TIMEFRAME_M5
    
    s_date = datetime.strptime(start_date, "%Y-%m-%d")
    s_year = s_date.year
    s_month = s_date.month
    s_day = s_date.day
    
    e_date = datetime.strptime(end_date, "%Y-%m-%d")
    e_year = e_date.year
    e_month = e_date.month
    e_day = e_date.day
    
    if start_date > end_date:
        print("Start date cannot be after end date")
        mt.shutdown()
        return None

    timezone = pytz.timezone("Etc/UTC")
    utc_from = datetime(s_year, s_month, s_day, tzinfo=timezone)
    utc_to = datetime(e_year, e_month, e_day, hour = 13, tzinfo=timezone)
    # get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 in UTC time zone
    rates = mt.copy_rates_range(symbol, timeframe, utc_from, utc_to)

    #rates = mt.copy_rates_range("XAUUSDm", timeframe, "2023-12-01", "2024-01-01")

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')

    mt.shutdown()
    return df

# Load data
data = get_15m_candles("XAUUSDm","2023-12-01", "2024-01-01")

In [20]:
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-12-01 00:00:00,2040.075,2040.858,2039.809,2040.858,263,200,0
1,2023-12-01 00:05:00,2040.883,2040.951,2040.368,2040.648,248,200,0
2,2023-12-01 00:10:00,2040.614,2041.401,2040.614,2041.048,281,200,0
3,2023-12-01 00:15:00,2041.035,2041.640,2041.035,2041.443,243,200,0
4,2023-12-01 00:20:00,2041.447,2041.939,2041.282,2041.598,255,200,0
...,...,...,...,...,...,...,...,...
5499,2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0
5500,2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0
5501,2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0
5502,2023-12-29 21:50:00,2062.657,2063.304,2062.583,2062.622,189,200,0


In [21]:
len(data)

5504

In [23]:
#data["VWAP"] = ta.vwap(data.high,data.low, data.close,data.tick_volume)
data["RSI"] = ta.rsi(data.close, length=16)
my_bbands = ta.bbands(data.close, length=14, std=2.0)
data=data.join(my_bbands)

In [17]:
data

,time,open,high,low,close,tick_volume,spread,real_volume,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0
0,2023-12-01 00:00:00,2040.075,2040.858,2039.809,2040.858,263,200,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-12-01 00:05:00,2040.883,2040.951,2040.368,2040.648,248,200,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-01 00:10:00,2040.614,2041.401,2040.614,2041.048,281,200,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-01 00:15:00,2041.035,2041.640,2041.035,2041.443,243,200,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01 00:20:00,2041.447,2041.939,2041.282,2041.598,255,200,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0,40.532406,2061.755458,2063.561286,2065.367114,0.175021,0.368402
5500,2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0,38.263050,2061.874294,2063.359214,2064.844135,0.143932,0.277694
5501,2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0,37.999171,2062.321219,2063.146000,2063.970781,0.079954,0.201739
5502,2023-12-29 21:50:00,2062.657,2063.304,2062.583,2062.622,189,200,0,37.801429,2062.396693,2063.035143,2063.673593,0.061894,0.176448


In [24]:
data['ATR']= ta.atr(data.high,data.low,data.close,length=7)


In [25]:
data

,time,open,high,low,close,tick_volume,spread,real_volume,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,ATR
0,2023-12-01 00:00:00,2040.075,2040.858,2039.809,2040.858,263,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-12-01 00:05:00,2040.883,2040.951,2040.368,2040.648,248,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-01 00:10:00,2040.614,2041.401,2040.614,2041.048,281,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-01 00:15:00,2041.035,2041.640,2041.035,2041.443,243,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01 00:20:00,2041.447,2041.939,2041.282,2041.598,255,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0,40.532406,2061.755458,2063.561286,2065.367114,0.175021,0.368402,0.713058
5500,2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0,38.263050,2061.874294,2063.359214,2064.844135,0.143932,0.277694,0.696050
5501,2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0,37.999171,2062.321219,2063.146000,2063.970781,0.079954,0.201739,0.671614
5502,2023-12-29 21:50:00,2062.657,2063.304,2062.583,2062.622,189,200,0,37.801429,2062.396693,2063.035143,2063.673593,0.061894,0.176448,0.678669


In [28]:
data["ATR"].min()

0.381236461395576

In [34]:
class Mystrat(Strategy):
    initsize=0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio= 1.5
        
        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()
                    
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.close[-1] -slatr
            tp1 = self.data.close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1,tp=tp1,size=self.mysize)
            
mystrat = Mystrat()           
bt = Backtest(data, mystrat, cash=100, margin=1/10,commission=0.00)
stat = bt.run()
stat
    

TypeError: Strategy.__init__() missing 3 required positional arguments: 'broker', 'data', and 'params'

In [39]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(data, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

ValueError: `data` must be a pandas.DataFrame with columns 'Open', 'High', 'Low', 'Close', and (optionally) 'Volume'

In [38]:
output

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                  68221.96986
Equity Peak [$]                   68991.21986
Return [%]                         582.219699
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   25.266427
Volatility (Ann.) [%]               38.383008
Sharpe Ratio                         0.658271
Sortino Ratio                        1.288779
Calmar Ratio                         0.763748
Max. Drawdown [%]                  -33.082172
Avg. Drawdown [%]                   -5.581506
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   94
Win Rate [%]                        54.255319
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

In [40]:
data


,time,open,high,low,close,tick_volume,spread,real_volume,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,ATR
0,2023-12-01 00:00:00,2040.075,2040.858,2039.809,2040.858,263,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-12-01 00:05:00,2040.883,2040.951,2040.368,2040.648,248,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-01 00:10:00,2040.614,2041.401,2040.614,2041.048,281,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-01 00:15:00,2041.035,2041.640,2041.035,2041.443,243,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01 00:20:00,2041.447,2041.939,2041.282,2041.598,255,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0,40.532406,2061.755458,2063.561286,2065.367114,0.175021,0.368402,0.713058
5500,2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0,38.263050,2061.874294,2063.359214,2064.844135,0.143932,0.277694,0.696050
5501,2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0,37.999171,2062.321219,2063.146000,2063.970781,0.079954,0.201739,0.671614
5502,2023-12-29 21:50:00,2062.657,2063.304,2062.583,2062.622,189,200,0,37.801429,2062.396693,2063.035143,2063.673593,0.061894,0.176448,0.678669


In [41]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [42]:
import yfinance as yf

In [55]:
df = get_15m_candles("XAUUSDm","2023-10-01", "2024-01-01")

In [56]:
df


,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-10-01 22:05:00,1847.284,1848.269,1847.052,1847.800,333,200,0
1,2023-10-01 22:10:00,1847.810,1848.611,1847.807,1848.122,216,200,0
2,2023-10-01 22:15:00,1848.156,1848.592,1848.022,1848.458,203,200,0
3,2023-10-01 22:20:00,1848.451,1848.678,1848.257,1848.504,173,200,0
4,2023-10-01 22:25:00,1848.504,1848.541,1847.847,1847.852,161,200,0
...,...,...,...,...,...,...,...,...
17559,2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0
17560,2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0
17561,2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0
17562,2023-12-29 21:50:00,2062.657,2063.304,2062.583,2062.622,189,200,0


In [57]:
df = df.set_index('time')

In [46]:
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-10-01 22:05:00,1847.284,1848.269,1847.052,1847.800,333,200,0
2023-10-01 22:10:00,1847.810,1848.611,1847.807,1848.122,216,200,0
2023-10-01 22:15:00,1848.156,1848.592,1848.022,1848.458,203,200,0
2023-10-01 22:20:00,1848.451,1848.678,1848.257,1848.504,173,200,0
2023-10-01 22:25:00,1848.504,1848.541,1847.847,1847.852,161,200,0
...,...,...,...,...,...,...,...
2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086,194,200,0
2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699,144,199,0
2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654,195,200,0


In [58]:
df = df.drop(columns=['tick_volume','spread','real_volume'])

In [59]:
df

,open,high,low,close
time,,,,
2023-10-01 22:05:00,1847.284,1848.269,1847.052,1847.800
2023-10-01 22:10:00,1847.810,1848.611,1847.807,1848.122
2023-10-01 22:15:00,1848.156,1848.592,1848.022,1848.458
2023-10-01 22:20:00,1848.451,1848.678,1848.257,1848.504
2023-10-01 22:25:00,1848.504,1848.541,1847.847,1847.852
...,...,...,...,...
2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086
2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699
2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654


In [60]:
df = df.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close'})

In [61]:
df

,Open,High,Low,Close
time,,,,
2023-10-01 22:05:00,1847.284,1848.269,1847.052,1847.800
2023-10-01 22:10:00,1847.810,1848.611,1847.807,1848.122
2023-10-01 22:15:00,1848.156,1848.592,1848.022,1848.458
2023-10-01 22:20:00,1848.451,1848.678,1848.257,1848.504
2023-10-01 22:25:00,1848.504,1848.541,1847.847,1847.852
...,...,...,...,...
2023-12-29 21:35:00,2063.202,2063.323,2062.668,2063.086
2023-12-29 21:40:00,2063.079,2063.221,2062.627,2062.699
2023-12-29 21:45:00,2062.643,2062.949,2062.424,2062.654


In [52]:
df

In [53]:
print(df)

None


In [62]:
bt = Backtest(df, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()


E:\Anaconda\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '15T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of values (2) does not match length of index (1)

In [63]:
output

Start                     2023-10-01 22:05:00
End                       2023-12-29 21:55:00
Duration                     88 days 23:50:00
Exposure Time [%]                   99.863357
Equity Final [$]                   2449.08583
Equity Peak [$]                       10000.0
Return [%]                         -75.509142
Buy & Hold Return [%]               11.637515
Return (Ann.) [%]                  -98.999132
Volatility (Ann.) [%]                 0.20926
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -75.517522
Avg. Drawdown [%]                  -75.517522
Max. Drawdown Duration       88 days 21:55:00
Avg. Drawdown Duration       88 days 21:55:00
# Trades                                  972
Win Rate [%]                         9.259259
Best Trade [%]                       1.707841
Worst Trade [%]                     -0.901611
Avg. Trade [%]                    

# Testing for Daily timeframe for last three years


In [66]:
def get_D_candles(symbol, start_date, end_date):
   
    if not mt.initialize():
        print("initialize() failed")
        mt.shutdown()
        return None

    timeframe = mt.TIMEFRAME_D1
    
    s_date = datetime.strptime(start_date, "%Y-%m-%d")
    s_year = s_date.year
    s_month = s_date.month
    s_day = s_date.day
    
    e_date = datetime.strptime(end_date, "%Y-%m-%d")
    e_year = e_date.year
    e_month = e_date.month
    e_day = e_date.day
    
    if start_date > end_date:
        print("Start date cannot be after end date")
        mt.shutdown()
        return None

    timezone = pytz.timezone("Etc/UTC")
    utc_from = datetime(s_year, s_month, s_day, tzinfo=timezone)
    utc_to = datetime(e_year, e_month, e_day, hour = 13, tzinfo=timezone)
    # get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 in UTC time zone
    rates = mt.copy_rates_range(symbol, timeframe, utc_from, utc_to)

    #rates = mt.copy_rates_range("XAUUSDm", timeframe, "2023-12-01", "2024-01-01")

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')

    mt.shutdown()
    return df

# Load data
ddf = get_D_candles("XAUUSDm","2020-01-01", "2024-01-01")

In [67]:
ddf

,time,open,high,low,close,tick_volume,spread,real_volume
0,2020-01-02,1518.464,1531.275,1517.172,1528.743,49227,260,0
1,2020-01-03,1528.725,1553.419,1528.725,1551.883,66506,260,0
2,2020-01-05,1577.360,1582.102,1566.855,1569.216,5727,260,0
3,2020-01-06,1569.151,1579.966,1560.818,1563.706,71962,260,0
4,2020-01-07,1563.723,1598.344,1555.277,1595.013,74597,260,0
...,...,...,...,...,...,...,...,...
1236,2023-12-26,2054.489,2069.044,2053.318,2065.502,59279,257,0
1237,2023-12-27,2065.846,2084.423,2061.468,2079.169,89029,199,0
1238,2023-12-28,2079.268,2088.485,2064.456,2066.564,100095,199,0
1239,2023-12-29,2066.602,2074.729,2058.095,2062.838,93900,199,0


## Arrange the data frame


In [68]:
ddf = ddf.set_index('time')
ddf = ddf.drop(columns=['tick_volume','spread','real_volume'])
ddf = ddf.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close'})


In [69]:
ddf

,Open,High,Low,Close
time,,,,
2020-01-02,1518.464,1531.275,1517.172,1528.743
2020-01-03,1528.725,1553.419,1528.725,1551.883
2020-01-05,1577.360,1582.102,1566.855,1569.216
2020-01-06,1569.151,1579.966,1560.818,1563.706
2020-01-07,1563.723,1598.344,1555.277,1595.013
...,...,...,...,...
2023-12-26,2054.489,2069.044,2053.318,2065.502
2023-12-27,2065.846,2084.423,2061.468,2079.169
2023-12-28,2079.268,2088.485,2064.456,2066.564


### Back test using Backtesting py library

In [70]:
bt = Backtest(ddf, SmaCross,
              cash=10000, commission=.00,
              exclusive_orders=True)

output = bt.run()

In [71]:
output

Start                     2020-01-02 00:00:00
End                       2024-01-01 00:00:00
Duration                   1460 days 00:00:00
Exposure Time [%]                   98.307816
Equity Final [$]                     9828.653
Equity Peak [$]                     11766.946
Return [%]                           -1.71347
Buy & Hold Return [%]               34.982989
Return (Ann.) [%]                   -0.350341
Volatility (Ann.) [%]               13.133104
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.323531
Avg. Drawdown [%]                   -5.951769
Max. Drawdown Duration      374 days 00:00:00
Avg. Drawdown Duration       84 days 00:00:00
# Trades                                   75
Win Rate [%]                        34.666667
Best Trade [%]                       16.07023
Worst Trade [%]                     -5.844364
Avg. Trade [%]                    